In [160]:
import arcpy

In [185]:
from arcgis.gis import GIS

In [186]:
import json

In [196]:
from arcgis.features import FeatureLayerCollection

In [201]:
from arcgis.features import FeatureLayer

In [10]:
import os

In [114]:
arcpy.env.overwriteOutput = True

In [2]:
camp_site = 'https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0'

In [3]:
path = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved\IllegalCampNotification_pro'

In [13]:
sdeFile = path + "\\RLIDDB.sde"

In [43]:
conn = arcpy.CreateDatabaseConnection_management(out_folder_path=path,
                                          out_name="RLIDDB.sde",
                                          database_platform="SQL_SERVER",
                                          instance="rliddb.int.lcog.org,5433",
                                          account_authentication="OPERATING_SYSTEM_AUTH",
                                          database="RLIDGeo")

In [41]:
if os.path.exists(sdeFile):
    os.remove(sdeFile)

In [44]:
arcpy.env.workspace = conn.getOutput(0)

In [205]:
fc = "RLIDGeo.DBO.Taxlot"

In [152]:
listdates1 = []

In [156]:
cursor = arcpy.da.SearchCursor(camp_site, "Date")

In [157]:
for row in cursor:
    listdates1.append(row)

In [62]:
from datetime import datetime

In [158]:
type(listdates1[0][0])

datetime.datetime

In [159]:
listdates1

[(datetime.datetime(2016, 5, 18, 15, 37),),
 (datetime.datetime(2016, 5, 18, 15, 38, 19, 867000),),
 (datetime.datetime(2016, 5, 18, 15, 39, 57, 341000),),
 (datetime.datetime(2016, 5, 16, 15, 42, 32, 964000),),
 (datetime.datetime(2016, 5, 16, 15, 43, 17, 842000),),
 (datetime.datetime(2016, 6, 2, 22, 45),),
 (datetime.datetime(2016, 6, 2, 22, 55, 14, 62000),),
 (datetime.datetime(2016, 6, 2, 23, 5, 52, 635000),),
 (datetime.datetime(2016, 6, 7, 23, 20),),
 (datetime.datetime(2016, 7, 9, 3, 16, 29, 449000),),
 (datetime.datetime(2016, 7, 19, 7, 0),),
 (datetime.datetime(2016, 7, 14, 19, 52, 29, 143000),),
 (datetime.datetime(2016, 7, 15, 15, 6, 19, 341000),),
 (datetime.datetime(2016, 7, 19, 21, 7, 1, 180001),),
 (datetime.datetime(2016, 7, 18, 21, 16, 14, 945000),),
 (datetime.datetime(2016, 7, 5, 21, 20, 18, 46001),),
 (datetime.datetime(2016, 7, 9, 21, 22, 44, 533000),),
 (datetime.datetime(2016, 7, 14, 21, 24, 46, 957001),),
 (datetime.datetime(2016, 7, 12, 21, 29, 29, 697000),),


In [68]:
listdates = [dt[0] for dt in listdates if dt[0] is not None]

In [69]:
maxdate = max(listdates)

In [70]:
maxdate

datetime.datetime(2023, 1, 18, 22, 52, 35, 515000)

In [71]:
maxdatestr = str(maxdate)

In [102]:
maxdatestr.split('.')[0]

'2023-01-18 22:52:35'

In [81]:
timestrlist = maxdatestr.split(' ')[1].split(':')

In [82]:
h = timestrlist[0]
m = timestrlist[1]
s = timestrlist[2].split('.')[0]

In [83]:
def convert_hour(str_h):
    h = int(str_h)
    if h >= 0 and h <= 11:
        res = [str_h, 'AM']
    elif h == 12:
        res = [str_h, 'PM']
    elif h > 12 and h <=23:
        res = [str(h-12), 'PM']
    return res

In [86]:
cvt_h = convert_hour(h)

In [94]:
def convert_date(datestr):
    datestrlist = datestr.split('-')
    Y = datestrlist[0]
    m = str(int(datestrlist[1]))
    d = str(int(datestrlist[2]))
    res = m+'/'+d+'/'+Y
    return res

In [95]:
convert_date(datestr)

'1/18/2023'

In [173]:
def getDateStr(date):
    def convert_hour(str_h):
        h = int(str_h)
        if h >= 0 and h <= 11:
            res = [str_h, 'AM']
        elif h == 12:
            res = [str_h, 'PM']
        elif h > 12 and h <=23:
            res = [str(h-12), 'PM']
        return res
    
    if date is None:
        res = None
    else:
        strdate = str(date)
        datestr = strdate.split(' ')[0]
        res = convert_date(datestr)
    return res

In [96]:
maxdatestr_n = convert_date(datestr) + ' '  + cvt_h[0] + ':' + m + ':' + s + ' ' + cvt_h[1]

In [97]:
maxdatestr_n

'1/18/2023 10:52:35 PM'

In [137]:
getDateStr(maxdate)

'1/18/2023 10:52:35 PM'

In [ ]:
Date = timestamp '2023-01-18 22:52:36'

In [100]:
f"Date = '{maxdatestr_n}'"

"Date = '1/18/2023 10:52:35 PM'"

In [129]:
arcpy.management.CopyFeatures(camp_site, path + '\\MyProject4.gdb\\datacopy')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\datacopy'>

In [134]:
arcpy.management.AddFields(path + '\\MyProject4.gdb\\datacopy', [['Date_str', "TEXT", 'datestr', 255, '1/18/2023 10:52:35 PM', '']])

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\datacopy'>

In [171]:
expression = "getDateStr(!Date!)"

codeblock = """
def getDateStr(date):
    def convert_date(datestr):
        datestrlist = datestr.split('-')
        Y = datestrlist[0]
        m = str(int(datestrlist[1]))
        d = str(int(datestrlist[2]))
        res = m+'/'+d+'/'+Y
        return res

    if date is None:
        res = None
    else:
        strdate = str(date)
        datestr = strdate.split(' ')[0]
        res = convert_date(datestr)
    return res"""

In [172]:
arcpy.management.CalculateField(path + '\\MyProject4.gdb\\datacopy', 'Date_str',  expression, "PYTHON3", 
                                codeblock)

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\datacopy'>

In [165]:
maxdatestr_n

'1/18/2023 10:52:35 PM'

In [169]:
maxdate

datetime.datetime(2023, 1, 18, 22, 52, 35, 515000)

In [174]:
getDateStr(maxdate)

'1/18/2023'

In [175]:
selres = arcpy.SelectLayerByAttribute_management(path + '\\MyProject4.gdb\\datacopy',
                                                 "NEW_SELECTION", f"Date_str = '{getDateStr(maxdate)}'")

In [125]:
selres = arcpy.SelectLayerByAttribute_management(camp_site, "NEW_SELECTION", "Date = date '2023-01-18'")

In [176]:
print(selres)

datacopy_Layer5


In [177]:
arcpy.management.CopyFeatures(selres, path + '\\MyProject4.gdb\\most_recent')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\most_recent'>

In [178]:
arcpy.analysis.SpatialJoin(selres, fc, path + '\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c', "JOIN_ONE_TO_ONE", "KEEP_ALL")

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c'>

In [182]:
arcpy.DeleteField_management(path + '\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c', 
                             ["Join_Count", "Unruly inhabitants"])

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c'>

In [183]:
arcpy.conversion.TableToExcel(path + '\\MyProject4.gdb\\HomelessCampSite_SpatialJoin14c', path+'\\most_recent.xlsx')

<Result 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\IllegalCampNotification_pro\\most_recent.xlsx'>